In [1]:
import jsondiff
import sqlite3
import json
import os
import psycopg2
from datetime import date, datetime
import csv

with open('./config.json') as f:
    config = json.load(f)

db_file = './sierra_patron_data.db'

class SQLiteConnection:
    """
    Usage:
    with SQLiteConnection('your_database_path.sqlite') as conn:
        # Your database operations here
    """
    def __init__(self, db_path):
        self.db_path = db_path

    @staticmethod
    def compute_diff(new_json, old_json):
        return str(
            jsondiff.diff(new_json, old_json, load=True, dump=True, marshal=True, syntax='explicit')
        )

    def __enter__(self):
        self.conn = sqlite3.connect(self.db_path)
        self.conn.create_function('json_diff', 2, self.compute_diff)
        return self.conn

    def __exit__(self, exc_type, exc_val, exc_tb):
        sql = """\
        PRAGMA analysis_limit=2000;
        PRAGMA optimize;
        """
        self.cursor = self.conn.cursor()
        self.cursor.executescript(sql)
        
        self.conn.close()

In [2]:
# # take a look at what the change data looks like

# sql = """\
# select
#     json_changes.json_table_name,
#     json_changes.diff,
#     json_changes.change_timestamp,
#     pj.json_data
# from
#     json_changes
#     join patron_address_json as pj on 
#         pj.patron_record_id = json_changes.patron_record_id
# """

# with SQLiteConnection(db_file) as con_sqlite: # \ 
# # psycopg2.connect(dsn=config['dsn']) as con, \
#     # cursor = con.cursor(name="named_cursor")
#     cursor_sqlite = con_sqlite.cursor()

#     cursor_sqlite.execute(sql)
#     columns = [col[0] for col in cursor_sqlite.description]

#     data = cursor_sqlite.fetchall()

# for row in data:
#     json_obj = json.loads(row[columns.index('json_data')])
#     print('current address:', json.dumps(json_obj, indent=4) )
#     print(
#         f"diff from {row[columns.index('change_timestamp')]}: {row[columns.index('diff')]}"
#     ) 
#     print('\n\n')
